<a href="https://colab.research.google.com/github/sayanbanerjee32/paddy_comp_with_jhoward/blob/main/paddy_comp_large_ensemble_tta_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

!pip install -Uq 'timm>=0.6.2.dev'
!pip install -Uq pynvml
from fastkaggle import *

     |████████████████████████████████| 61 kB 2.5 MB/s 
     |████████████████████████████████| 497 kB 7.3 MB/s 
     |████████████████████████████████| 46 kB 2.9 MB/s 


In [2]:
# import colab libraries
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/Kaggle_api_auth/kaggle.json ~/.kaggle/

In [4]:
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
comp = 'paddy-disease-classification'
path = setup_comp(comp, install='fastai "timm>=0.6.2.dev"')

100%|██████████| 1.02G/1.02G [00:09<00:00, 120MB/s] 


In [6]:
from fastai.vision.all import *
set_seed(42)
tst_files = get_image_files(path/'test_images').sorted()
path.ls()

(#4) [Path('paddy-disease-classification/train_images'),Path('paddy-disease-classification/train.csv'),Path('paddy-disease-classification/test_images'),Path('paddy-disease-classification/sample_submission.csv')]

In [7]:
df = pd.read_csv(path/'train.csv')
df.label.value_counts()

normal                      1764
blast                       1738
hispa                       1594
dead_heart                  1442
tungro                      1088
brown_spot                   965
downy_mildew                 620
bacterial_leaf_blight        479
bacterial_leaf_streak        380
bacterial_panicle_blight     337
Name: label, dtype: int64

In [8]:
trn_path = path/'train_images'/'bacterial_panicle_blight'

In [9]:
def train(arch, size, item=Resize(480, method='squish'), accum=1, finetune=True, epochs=12):
    dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=item,
        batch_tfms=aug_transforms(size=size, min_scale=0.75), bs=64//accum)
    cbs = GradientAccumulation(64) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    if finetune:
        learn.fine_tune(epochs, 0.01)
        return learn.tta(dl=dls.test_dl(tst_files))
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs, 0.01)

In [10]:
import gc
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [11]:
train('convnext_small_in22k', 128, epochs=1, accum=2, finetune=False)
report_gpu()

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_small_22k_224.pth" to /root/.cache/torch/hub/checkpoints/convnext_small_22k_224.pth


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:05


GPU:0
process       2524 uses     1961.000 MB GPU memory


In [12]:
train('convnext_small_in22k', 128, epochs=1, accum=4, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:04


GPU:0
process       2524 uses     2489.000 MB GPU memory


In [13]:
train('convnext_large_in22k', 224, epochs=1, accum=2, finetune=False)
report_gpu()

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_224.pth" to /root/.cache/torch/hub/checkpoints/convnext_large_22k_224.pth


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:15


GPU:0
process       2524 uses     4665.000 MB GPU memory


In [14]:
train('convnext_large_in22k', (320,240), epochs=1, accum=2, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:21


GPU:0
process       2524 uses     4985.000 MB GPU memory


In [15]:
train('vit_large_patch16_224', 224, epochs=1, accum=2, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:18


GPU:0
process       2524 uses     6565.000 MB GPU memory


In [16]:
train('swinv2_large_window12_192_22k', 192, epochs=1, accum=2, finetune=False)
report_gpu()

/usr/local/lib/python3.7/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v2.0.0/swinv2_large_patch4_window12_192_22k.pth" to /root/.cache/torch/hub/checkpoints/swinv2_large_patch4_window12_192_22k.pth


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:12


GPU:0
process       2524 uses     4899.000 MB GPU memory


In [17]:
train('swin_large_patch4_window7_224', 224, epochs=1, accum=2, finetune=False)
report_gpu()

Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_patch4_window7_224_22kto1k.pth" to /root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:12


GPU:0
process       2524 uses    11729.000 MB GPU memory


### Running the model

In [18]:
res = 640,480
trn_path = path/'train_images'

In [19]:
models = {
    'convnext_large_in22k': {
        (Resize(res), (320,224)),
    }, 'vit_large_patch16_224': {
        (Resize(480, method='squish'), 224),
        (Resize(res), 224),
    }, 'swinv2_large_window12_192_22k': {
        (Resize(480, method='squish'), 192),
        (Resize(res), 192),
    }, 'swin_large_patch4_window7_224': {
        (Resize(res), 224),
    }
}

In [20]:
tta_res = []

for arch,details in models.items():
    for item,size in details:
        print('---',arch)
        print(size)
        print(item.name)
        tta_res.append(train(arch, size, item=item, accum=2)) #, epochs=1))
        gc.collect()
        torch.cuda.empty_cache()

--- convnext_large_in22k
(320, 224)
Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (2, 0), 'p': 1.0}


epoch,train_loss,valid_loss,error_rate,time
0,1.053776,0.716343,0.205190,05:28


epoch,train_loss,valid_loss,error_rate,time
0,0.433099,0.326482,0.087938,07:31
1,0.343719,0.318870,0.094666,07:30
2,0.347849,0.337595,0.084575,07:30
3,0.281716,0.224047,0.061989,07:30
4,0.204833,0.206657,0.048534,07:30
5,0.149418,0.172178,0.039885,07:31
6,0.159423,0.216742,0.052379,07:30
7,0.100746,0.158224,0.033157,07:30
8,0.062867,0.154552,0.031716,07:31
9,0.056446,0.133801,0.024988,07:31


--- vit_large_patch16_224
224
Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (2, 0), 'p': 1.0}


epoch,train_loss,valid_loss,error_rate,time
0,1.060387,0.650201,0.193657,06:21


...

epoch,train_loss,valid_loss,error_rate,time
0,1.081327,0.632106,0.211917,04:29


epoch,train_loss,valid_loss,error_rate,time
0,0.438844,0.262224,0.083133,05:43
1,0.351298,0.265155,0.085055,05:40
2,0.336326,0.290028,0.083133,05:38
3,0.295138,0.366627,0.095627,05:38
4,0.221481,0.204586,0.056704,05:37
5,0.176786,0.216246,0.050937,05:37
6,0.134483,0.136939,0.034118,05:38
7,0.095112,0.139158,0.035079,05:37
8,0.060815,0.142062,0.032196,05:38
9,0.055074,0.121160,0.025949,05:37


--- swin_large_patch4_window7_224
224
Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (2, 0), 'p': 1.0}


epoch,train_loss,valid_loss,error_rate,time
0,1.155230,0.663505,0.210956,04:32


epoch,train_loss,valid_loss,error_rate,time
0,0.540818,0.349656,0.113407,05:59
1,0.394694,0.249993,0.075925,05:59
2,0.380094,0.315201,0.098510,05:59
3,0.290141,0.193133,0.060548,05:59
4,0.230833,0.165887,0.044690,05:59
5,0.197704,0.164461,0.044690,06:00
6,0.161300,0.120706,0.037482,05:59
7,0.097327,0.124061,0.028352,05:59
8,0.074834,0.121047,0.025469,06:00
9,0.063759,0.102832,0.024507,05:59


### Ensembling

In [21]:
tta_prs = first(zip(*tta_res))
tta_prs += tta_prs[1:3]

In [22]:
avg_pr = torch.stack(tta_prs).mean(0)
avg_pr.shape

torch.Size([3469, 10])

### Submission

In [23]:
ss = pd.read_csv(path/'sample_submission.csv')
ss.head()

,image_id,label
0,200001.jpg,NaN
1,200002.jpg,NaN
2,200003.jpg,NaN
3,200004.jpg,NaN
4,200005.jpg,NaN


In [24]:
dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=Resize(480, method='squish'),
    batch_tfms=aug_transforms(size=224, min_scale=0.75))
tst_dl = dls.test_dl(tst_files)

In [25]:
idxs = avg_pr.argmax(dim=1)
vocab = np.array(dls.vocab)
ss['label'] = vocab[idxs]

In [26]:
ss.to_csv('subm.csv', index=False)
!head subm.csv

image_id,label
200001.jpg,hispa
200002.jpg,normal
200003.jpg,blast
200004.jpg,blast
200005.jpg,blast
200006.jpg,brown_spot
200007.jpg,dead_heart
200008.jpg,brown_spot
200009.jpg,hispa


In [27]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('subm.csv', 'ensemble 224 large tta', comp)

100%|██████████| 70.4k/70.4k [00:03<00:00, 22.7kB/s]


In [ ]:
if not iskaggle:
    push_notebook('saan', 'paddy-initial-model',
                  title='Paddy initial model',
                  file='/content/drive/MyDrive/Colab Notebooks/paddy_comp_initial_model.ipynb',
                  competition=comp, private=True, gpu=True)

Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/code/saansd2003/paddy-initial-model
